In [ ]:
#Imports and load data
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import * # Needed for Filters like When, Like etc.

spark = SparkSession.builder.appName("penguins").master("local[4]").getOrCreate()
df_penguins = spark.read.csv("data/penguins.csv",header=True,inferSchema=True)
spark.sparkContext.setLogLevel("ERROR")


In [ ]:
df_penguins.show(3)

In [ ]:
# create indexer object
indexer = StringIndexer(inputCols=["sex","island","species"], outputCols=["sex_index","island_index","species_index"], handleInvalid='error', stringOrderType='frequencyDesc')

# Fit the indexer on the data and transform the data
df_penguins_indexed = indexer.fit(df_penguins).transform(df_penguins)

df_clean = df_penguins_indexed.select(
    "bill_length_mm",
    "bill_depth_mm",
    "flipper_length_mm",
    "body_mass_g",
    "sex_index",
    "island_index",
    "species_index")



In [ ]:
df_clean.printSchema()

In [ ]:
df_penguins_indexed.write.save("tmp.json",format="json")

In [ ]:
encoder = OneHotEncoder(inputCols=["sex_index"], outputCols=["sex_onehot"])
data = encoder.fit(df_clean).transform(df_clean)
data.head(30)

In [ ]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
assembler = [VectorAssembler(inputCols=[col], outputCol=col+'_vec') for col in ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
scale = [StandardScaler(inputCol=col+'_vec', outputCol=col+'_scaled') for col in ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
pipe = Pipeline(stages = assembler + scale)
df_scale = pipe.fit(df_trans).transform(df_trans)